# Task 5a - Ising model

### Goals:
- check the onset of spontaneous magnetisation in the Ising model
- find the critical temperature
- check the discontinuity of the response properties at $T_c$

# Ising model

Finally, we demonstrate the application of Metropolis algorithm in one of the simplest phase change prototypes: a 2-dimensional Ising model.
In Ising model, we consider a 2D lattice, and each lattice site is assigned to a configuration $s_i,$ which is a discrete variavle that can be either $+1$ or $-1$.
The Hamiltonian of the model reads
$$H=-J \sum_{\langle i j\rangle} s_i s_j-\mu h \sum_j s_j,$$
where J is the exchange energy, $\mu$ is the atomic magnetic moment, and $h$ represents and external field.
For simplicity, we set $J$ to unity and $h$ to zero (i.e. no magnetic field).
The first summation in the Hamiltonian is over all pairs of *nearest neighbours* on the lattice.
If we take the variable $s$ as a spin value, then the model is *ferromagnetic* if every lattice site has the same $s$ value, or *antiferromagnetic* if every two neighboring lattice sites have different spin values.
The sign of the exchange energy $J$ determines if the system favors a ferromagnetic configuration or an antiferromagnetic one.
For example, if $J>0$, a configuration with two pairing neighbours having parallel spin is favoured (ferromagnetism).



Given the Hamiltonian above, we have the partition function $Z$
$$Z=\sum_{\text {configs }} e^{-H(s) / k_B T},$$
where $k_B$ is Boltzmann's constant, and $T$ is temperature.
From the partition function, we might derive many properties such as free energy.
We are not going through de details here.


In this task, we're going to study the 2D Ising model using the Monte Carlo method.

As a first step, a 2D lattice is initialized with some starting spin configuration.
The spin configuration can be fully parallel, and this gives rise to a high net spin value.
Here we introduce the **magnetization** $M:$
$$M=\frac{1}{N}\sum_i s_i,$$
where $N$ is the number of lattice sites.
A parallel configuration leads to $M=\pm 1.$
This is the case at low temperatures.
On the other hand, if the spin configuration is anti-aligned, then $M=0.$
This corrisponds to a high temperature configuration.
In between, when the environment is 'warm', spin values are randomly selected over the lattice sites.

In the second step, we calculate the energy $E$ of the initial configuration
$$E=-\frac{1}{2} \sum_i \sum_{j \in \mathrm{NN}} s_i s_j,$$
where 1/2 accounts for double counting in the summation and NN stands for nearest neighbour sites around site $i.$
Periodic boundary conditions should be taken into account.

Next, the spin of a random lattice site is flipped.
This hives a new trial spin configuration.
We then calculate the new energy $E_\text{new}$ and we accept the new configuration if the new energy is lower than the new one (i.e. $\Delta E = E_{\text{new}}-E_{\text{old}} \leq 0$).
Otherwise, we compare the probability of the Boltzmann factor $e^{-\Delta E/kT}$ to a random number $\eta\in (0,1).$
If the probability is larger, then the new configuration is accepted, otherwise the new configuration is rejected and the old one will be used as a starting point in the following iteration.

After a number of Monte Carlo steps are performed, you can get the impression of the evolution of the spin configuration and magnetization of the Ising model.
Recall that the Boltzmann factor is temperature dependent, so low $T$ and high $T$ will result in very different magnetisation bahaviour after a long run.




In [ ]:
#!/usr/bin/env python

import ising
import numpy as np
import time
import matplotlib.pylab as plt
from IPython.display import HTML
from scipy.optimize import curve_fit

## Spontaneous magnetisation

Perform the Monte Carlo simulation of the Ising model for different initial spin configurations (`hot`, `cold ` or `warm`).

In your simulation pay close attention to the changes in the magnetisation $M$ along the Monte Carlo steps.

Does a low temperature give a spontanteous magnetisation $(M=\pm 1)$?

What happens if the temperature is very high?

<div class="alert alert-block alert-info"><b>TODO:</b> Perform the Monte Carlo simulation of the Ising model for different initial spin configurations (`hot`, `cold ` or `warm`) and different temperatures and compare the results.
 </div>

In [ ]:

# ---------------------------------------------------------------------

ngrid = 20              # we use a 10x10grid 
neq = 100              # Number of equilibration step (multipled by nskip)
nstep= 1500             # Number of sampling
nskip = 5              # number of steps to be skipped between two samplings
init_status = 'warm'   # initial status of the spin configuration
kT = 0.8               # temperature
scheme = 'random'        # flip scheme: either ’random’ or ’scan’
verbose = 0            # 1 = enable, 0 = disable output


calc  = ising.ising(ngrid, nstep, nskip, neq, init_status, kT, scheme)   # init the ising object
calc.run(verbose)               # self-explanatory



In [ ]:

# Show an animation of the spins
# it might take some time and fill the available memory
# try to keep the number of frames low by stetting an appropriate value
# for `plot_every`

anim = calc.animation_jupyter(plot_every=30)

# Display the animation in the notebook
from IPython.display import HTML

#set matplotlib in interactive mode, needed if using jupyter
%matplotlib notebook 
HTML(anim.to_jshtml())

In [ ]:
# Or alternatively just show a single frame (faster):

# re-set normal matplotlib backend
%matplotlib inline

calc.plot_step(calc.nstep)
plt.show()

In [ ]:
# Plot the magnetization
plt.figure()
plt.plot(calc.arrM)
plt.ylabel('M')
plt.xlabel('step')
plt.show()

## Critical temperature

**Locate the temperature $T_c$ at which a phase change takes place.**
You can get an idea of what $T_c$ is by investigating how the modulus of the magnetisation $|M|$ at the end of the simulation varies with $kT$.

<div class="alert alert-block alert-info"><b>TODO:</b> Run a series of simulations to show the behaviour of |M| as a function of kT </div>

In [ ]:
# Your code here

## Response properties

The specific head density $C_v$ can be expressed in terms of the variance of the energy
$$C_v=\left(\left\langle E^2\right\rangle-\langle E\rangle^2\right) / k T.$$

In a similar way, the magnetic susceptibility $\chi$ can ve obtained from the variance of the magnetisation (fluctuation-dissipation theorem):
$$ \chi=\left(\left\langle M^2\right\rangle-\langle M\rangle^2\right) / k T. $$

Several independent configuration runs are needed to perform an ensemble average for these two parameters.
For each temperature, make sure that thermal equilibrium has been reached, which might take a long time using single-flip (`scheme=random`) Metropolis scheme.
You can use `scheme=scan` for faster equilibration.

<div class="alert alert-block alert-info"><b>TODO:</b> Generate a plot of \chi and Cv vs kT </div>

The simulation takes some time; you can find an example of the results in the files `ising_chi_example.dat` and `ising_heat_example.dat`


In [ ]:

# ---------------------------------------------------------------------
# susceptibility

ngrid = 16
nstep = 1000
nskip = ngrid**2
neq = 200
init_status = 'hot'
kT = 2.29
verbose = 0
scheme = 'scan' 
tmin = 1.5
tmax = 3.2
tp = 30            # number of temperatures
m = np.zeros((tp,3))
m[:,0] = np.linspace(tmin,tmax,tp)  # list of temperatures to be investigated

nrep = 15 
t_start = time.time()

for i in range(tp):
   print(f"T = {m[i,0]:4.2f} ({i+1}/{tp})")
   print('Replica = ',end='')
   for j in range(nrep):
       print(f'{j} ',end='')
       calc = ising.ising(ngrid, nstep, nskip, neq, init_status, m[i,0], scheme)
       calc.run(verbose)
       m[i,1] += np.abs(calc.mavg)
       m[i,2] += (calc.m2avg - calc.mavg**2)/m[i,0]
   print("Time: {0:6.2f}s".format(time.time()-t_start))

m[:,1] /= nrep            # <M>
m[:,2] /= nrep         # chi  

np.savetxt('ising_chi.dat', m)    


Adapt the code above to find $C_v$.

You can retrieve $\langle E^2 \rangle$ and $\langle E \rangle$ from the `calc.E02avg` and `calc.E0avg`.

In [ ]:
# Specific heat

# your code here



In [ ]:
data=np.loadtxt('ising_heat_example.dat')

plt.plot(data[:,0],data[:,2]/calc.ngrid, marker='x', lw=.5)
plt.xlabel('kT')
plt.ylabel('chi')